Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# 3. Uczenie nienadzorowane

W poprzednim zeszycie zbadaliśmy różne modele grafowych sieci neuronowych w scenariuszu nadzorowanej klasyfikacji wierzchołków. Teraz zajmiemy się tematem uczenia nienadzorowanego dla GNNów, który jak już wcześniej wspomnieliśmy, jest trochę bardziej złożony.

W jaki sposób powinna być skonstruowana funkcja kosztu? Czy możemy zastosować model autokodera? Jak w takim razie powinien działać dekoder? Jak uwzględnić relacje między wierzchołkami?

To tylko kilka pytań, na które należy odpowiedzieć podczas opracowania nienadzorowanego modelu grafowych sieci neuronowych. W ostatnich latach powstało wiele rozwiązań, obejmujących między innymi:
- grafowe autokodery (w tym wariacyjne)
- uczenie kontrastowe
- uczenie samo-nadzorowane

W ninejszym zeszycie najpierw zbadamy **model grafowego autokodera** jako najprostszego modelu stosowanego w nienadzorowanym uczeniu reprezentacji grafów, a następnie zbadamy **model Graph Barlow Twins**.

## 3.1. Grafowy autokoder
W 2016 roku Kipf, autor pracy wprowadzającej architekturę GCN, opublikował również [artykuł](https://arxiv.org/pdf/1611.07308.pdf) w którym pokazał jak wykorzystać GCNa (lub dowolny inny GNNowy model) w znanej nam architekturze autokodera. Jak wiemy taki model składa się z dwóch komponentów:
- **kodera** - w tym wypadku koderem jest wybrana przez nas grafowa sieć neuronowa

$$\mathbf{Z} = \text{GNN}(\mathbf{X}, \mathbf{A})$$

- **dekodera** - model dekodera na wejściu przyjmuje wyznaczone reprezentacje $\mathbf{Z}$, a na wyjściu oblicza rekonstrukcję danego obiektu, w naszym przypadku grafu. Jednak co dokładnie powinien odtworzyć taki grafowy dekoder? Strukturę, atrybuty, czy jedno i drugie? W swojej pracy Kipf zaproponował, aby skupić się wyłącznie na strukturze grafu, tzn. dokonać rekonstrukcji krawędzi. Taki wariant również będziemy rozważać na cele tego laboratorium, przy czym inne scenariusze są równie poprawne i w zależności od konkretnego zadania mogą dostarczać lepszych wyników. 

W celu zbudowania odpowiedniego dekodera strukturalnego musimy określić w jaki sposób będziemy decydować czy istnieje krawędź między parą dowolnych wierzchołków. Najpopularniejszym rozwiązaniem jest wykorzystanie iloczynu skalarnego, podobnie jak w przypadku modelu Node2vec definiowaliśmy podobieństwo wierzchołków w przestrzeni reprezentacji. Tutaj wybór ten jest umotywowany intuicją, że podobne wierzchołki powinny być połączone krawędzią. Dekoder ma zatem postać:

$$\hat{\mathbf{A}} = \sigma(\mathbf{Z}\mathbf{Z}^T),$$

gdzie:
- $\hat{\mathbf{A}}$ to rekonstrukcja macierzy sąsiedztwa
- $\sigma(\cdot)$ to sigmoidalna funkcja aktywacji

## Zadanie 3.1. (4 pkt)
Wykorzystując zaimplementowane w PyTorch-Geometricu modele:
- grafowego autokodera `GAE` - [link](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.GAE.html)
- dekodera iloczynu skalarnego `InnerProductDecoder` - [link](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.InnerProductDecoder.html)

dokończ implementację klasy `BaseUnsupervisedGNN` oraz `GraphAutoencoder`. Zastosuj się do komentarzy umieszczonych przy odpowiednich funkcjach.

In [ ]:
import pytorch_lightning as pl
import torch
from sklearn.metrics import roc_auc_score
from torch import nn
from torch_geometric.data import Data

class BaseUnsupervisedGNN(pl.LightningModule):
    """Base class for unsupervised GNN models for node representations."""

    def __init__(self):
        super().__init__()
        
        self._downstream_model = None
        self.training_step_outputs = []
    
    def on_train_batch_end(self, outputs, batch, batch_idx) -> None:
        self.training_step_outputs.append(outputs)
    
    def on_validation_epoch_start(self) -> None:
        z_train = torch.cat([out["z_train"].detach() for out in self.training_step_outputs], dim=0)
        y_train = torch.cat([out["y_train"] for out in self.training_step_outputs], dim=0)
        z_train, y_train = z_train.cpu(), y_train.cpu()
        self.training_step_output = []
        
        auc = ...

        # TODO: Naucz model regresji logistycznej na parach (`z_train`, `y_train`), 
        # a następnie oblicz wartość miary AUC na zbiorze treningowym. Wykorzystaj
        # model regresji logistycznej z biblioteki Scikit-Learn i przypisz go do
        # pola: `self._downstream_model`.
        
        # TU WPISZ KOD
        raise NotImplementedError()

        self.log("train/auc", auc, on_epoch=True, on_step=False)

    def validation_step(self, batch: Data, batch_idx: int):
        auc = self._compute_auc(data=batch, mask=batch.val_mask)

        self.log("step", self.trainer.current_epoch)
        self.log("val/auc", auc, on_epoch=True, on_step=False)

        return {"auc": auc}

    def test_step(self, batch: Data, batch_idx: int):
        auc = self._compute_auc(data=batch, mask=batch.test_mask)

        self.log("step", self.trainer.current_epoch)
        self.log("test/auc", auc, on_epoch=True, on_step=False)

        return {"auc": auc}

    def predict_step(
        self,
        batch: Data,
        batch_idx: int,
        dataloader_idx: int | None = None,
    ) -> tuple[torch.Tensor, torch.Tensor]:
        z = self(batch.x, batch.edge_index)
        y = batch.y

        return z, y

    def _compute_auc(self, data: Data, mask: torch.Tensor) -> float:
        # TODO: Oblicz wartość miary AUC dla zadanego przez maskę
        # podzbioru wierzchołków.
        
        auc = ...
        # TU WPISZ KOD
        raise NotImplementedError()

        return auc

    def configure_optimizers(self):
        return torch.optim.AdamW(
            params=self.parameters(),
            lr=1e-3,
            weight_decay=5e-4,
        )

In [ ]:
class GraphAutoencoder(BaseUnsupervisedGNN):
    """Unsupervised graph autoencoder for node representations."""

    def __init__(self, gnn: nn.Module):
        super().__init__()

        # TODO: Utwórz model GAE
        
        # TU WPISZ KOD
        raise NotImplementedError()

    def forward(
        self,
        x: torch.Tensor,
        edge_index: torch.Tensor,
    ) -> torch.Tensor:
        # TODO: Funkcja forward powinna zwracać wektory reprezentacji `z`
        
        # TU WPISZ KOD
        raise NotImplementedError()

    def training_step(self, batch: Data, batch_idx: int) -> dict:
        z = ...
        loss = ...

        # TODO: Wyznacz wektory reprezentacji `z` oraz oblicz funkcję kosztu `loss`
        
        # TU WPISZ KOD
        raise NotImplementedError()

        self.log("step", self.trainer.current_epoch)
        self.log("train/loss", loss.item(), on_epoch=True, on_step=False)

        return {
            "loss": loss,
            "z_train": z[batch.train_mask],
            "y_train": batch.y[batch.train_mask],
        }

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.data.lightning import LightningNodeData

dataset = Planetoid(root="./data", name="Cora")

datamodule = LightningNodeData(
    data=dataset[0],
    loader="full",
)

Dla wszystkich modeli zdefiniujmy sobie zbiór wspólnych hiperparametrów:

In [ ]:
hparams = {
    "num_epochs": 10,
    "hidden_dim":  256,
    "emb_dim": 128,
}
ACCELERATOR = "cpu" # change to "cuda" in order to use GPU

Porównamy teraz jakość działania modeli GNNowych, wprowadzonych w poprzednim zeszycie:

In [ ]:
from torch import nn
from torch_geometric.nn import GATConv, GCNConv, SAGEConv


class GCNEncoder(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z


class GraphSAGEEncoder(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z


class GATEncoder(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=1)
        self.act1 = nn.ReLU()
        self.conv2 = GATConv(hidden_dim, out_dim, heads=1)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z

In [ ]:
import matplotlib.pyplot as plt

from src.trainer import get_default_trainer
from src.utils import visualize_embeddings


def evaluate_unsupervised_models():
    scenarios = [
        ("GCN", GCNEncoder),
        ("GraphSAGE", GraphSAGEEncoder),
        ("GAT", GATEncoder),
    ]
    
    for model_name, gnn_cls in scenarios:
        gnn = gnn_cls(
            in_dim=datamodule.data.num_node_features,
            hidden_dim=hparams["hidden_dim"],
            out_dim=hparams["emb_dim"],
        )
    
        model = GraphAutoencoder(gnn=gnn)

        trainer = get_default_trainer(
            num_epochs=hparams["num_epochs"],
            model_name=f"gae_{model_name}",
            accelerator=ACCELERATOR,
        )
    
        trainer.fit(model=model, datamodule=datamodule)
    
        test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc"]
        z, y = trainer.predict(model=model, datamodule=datamodule)[0]
        z, y = z.cpu(), y.cpu()
    
        fig = visualize_embeddings(z=z, y=y)
        fig.suptitle(f"GAE {model_name} - test AUC: {test_auc * 100.:.2f} [%]")
    
        plt.show()
    

evaluate_unsupervised_models()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./data/logs/ --port 6006

## 3.2. Graph Barlow Twins

Grafowe autokodery są obecnie już rzadko stosowane i zostały zastąpione przez modele należące do metod samonadzorowanych (ang. *self-supervised*). Jednym z przykładów jest model stanowiący rozszerzenie modelu Barlow Twins na dziedzinę grafów – [**Graph Barlow Twins**](https://arxiv.org/abs/2106.02466). Potok przetwarzania został przedstawiony na rysunku poniżej:

![](./assets/graph-barlow-twins.png)

Dla zadanego grafu wejściowego $\mathcal{G}$ tworzone są dwa zmodyfikowane widoki $\mathcal{G}^{(1)}, \mathcal{G}^{(2)}$ za pomocą funkcji augmentacji (tutaj: losowe usuwanie krawędzi oraz maskowanie atrybutów wierzchołków). Następnie oba widoki są przetwarzane przez ten sam moduł kodera GNN, w wyniku czego otrzymujemy dwa zestawy (dwie macierze) reprezentacji wierzchołków $Z^{(1)}, Z^{(2)}$. Dla tych macierzy liczymy macierz korelacji wzajemnej $\mathcal{C}$:

$$\mathcal{C}_{ij} = \frac{\sum_b Z^{(1)}_{b,i} Z^{(2)}_{b,j}}{\sqrt{\sum_b (Z^{(1)}_{b,i})^2} \sqrt{\sum_b (Z^{(2)}_{b,j})^2}} $$

Model jest uczony za pomocą funkcji kosztu, która zbliża wartości w macierzy korelacji wzajemnej do macierzy jednostkowej:

$$\mathcal{L}_\text{BT} = \sum_i (1 - C_{ii})^2 + \lambda\sum_i\sum_{j \ne i} C_{ij}^2$$

## Zadanie 3.2 (1 pkt)
Dokończ implementację modelu Graph Barlow Twins

In [ ]:
import torch
from torch_geometric.data import Data


class GraphAugmentor:
    """Masks node features (same for all nodes) and drops edges."""

    def __init__(self, p_x: float, p_e: float):
        self._p_x = p_x
        self._p_e = p_e

    def __call__(self, data: Data):
        x_a = mask_features(data.x, p=self._p_x)
        x_b = mask_features(data.x, p=self._p_x)

        edge_index_a = drop_edges(data.edge_index, p=self._p_e)
        edge_index_b = drop_edges(data.edge_index, p=self._p_e)

        return (x_a, edge_index_a), (x_b, edge_index_b)


def mask_features(x: torch.Tensor, p: float) -> torch.Tensor:
    num_features = x.size(-1)
    device = x.device

    return bernoulli_mask(size=(1, num_features), prob=p).to(device) * x


def drop_edges(edge_index: torch.Tensor, p: float) -> torch.Tensor:
    num_edges = edge_index.size(-1)
    device = edge_index.device

    mask = bernoulli_mask(size=num_edges, prob=p).to(device) == 1.

    return edge_index[:, mask]


def bernoulli_mask(size: int | tuple[int, ...], prob: float):
    return torch.bernoulli((1 - prob) * torch.ones(size))

In [ ]:
import torch

EPS = 1e-15


def barlow_twins_loss(
    z_a: torch.Tensor,
    z_b: torch.Tensor,
) -> torch.Tensor:
    batch_size = z_a.size(0)
    feature_dim = z_a.size(1)
    _lambda = 1 / feature_dim

    # Apply batch normalization
    z_a_norm = (z_a - z_a.mean(dim=0)) / (z_a.std(dim=0) + EPS)
    z_b_norm = (z_b - z_b.mean(dim=0)) / (z_b.std(dim=0) + EPS)

    # Cross-correlation matrix
    c = (z_a_norm.T @ z_b_norm) / batch_size

    # Loss function
    off_diagonal_mask = ~torch.eye(feature_dim).bool()
    loss = (
        (1 - c.diagonal()).pow(2).sum()
        + _lambda * c[off_diagonal_mask].pow(2).sum()
    )

    return loss

In [ ]:
class GCNEncoderBN(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim, momentum=0.01)  # same as `weight_decay = 0.99`
        self.act1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.bn1(self.conv1(x, edge_index)))
        z = self.act2(self.conv2(z, edge_index))
        return z


class GraphSAGEEncoderBN(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim, momentum=0.01)  # same as `weight_decay = 0.99`
        self.act1 = nn.ReLU()
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.bn1(self.conv1(x, edge_index)))
        z = self.act2(self.conv2(z, edge_index))
        return z


class GATEncoderBN(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim, momentum=0.01)  # same as `weight_decay = 0.99`
        self.act1 = nn.ReLU()
        self.conv2 = GATConv(hidden_dim, out_dim, heads=1)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.bn1(self.conv1(x, edge_index)))
        z = self.act2(self.conv2(z, edge_index))
        return z

In [ ]:
class GraphBarlowTwins(BaseUnsupervisedGNN):
    """Self-supervised Barlow Twins model for graphs."""

    def __init__(self, encoder: nn.Module, p_x: float, p_e: float):
        super().__init__()

        self.augmentor = GraphAugmentor(p_x=p_x, p_e=p_e)
        self.encoder = encoder

    def forward(
        self,
        x: torch.Tensor,
        edge_index: torch.Tensor,
    ) -> torch.Tensor:
        return self.encoder(x, edge_index)

    def training_step(self, batch: Data, batch_idx: int) -> dict:
        # Uzupełnij potok przetwarzania modelu GBT, aby obliczyć wartość funkcji kosztu `loss`

        loss = ...
        # TU WPISZ KOD
        raise NotImplementedError()
        
        self.log("step", self.trainer.current_epoch)
        self.log("train/loss", loss.item(), on_epoch=True, on_step=False)

        z = self(x=batch.x, edge_index=batch.edge_index) 
        
        return {
            "loss": loss,
            "z_train": z[batch.train_mask],
            "y_train": batch.y[batch.train_mask],
        }

In [ ]:
def evaluate_gbt_models():
    scenarios = [
        ("GCN", GCNEncoderBN),
        ("GraphSAGE", GraphSAGEEncoderBN),
        ("GAT", GATEncoderBN),
    ]
    
    for model_name, gnn_cls in scenarios:
        gnn = gnn_cls(
            in_dim=datamodule.data.num_node_features,
            hidden_dim=hparams["hidden_dim"],
            out_dim=hparams["emb_dim"],
        )
    
        model = GraphBarlowTwins(encoder=gnn, p_x=0.2, p_e=0.4)

        trainer = get_default_trainer(
            num_epochs=hparams["num_epochs"],
            model_name=f"gbt_base_{model_name}",
            accelerator=ACCELERATOR,
        )
    
        trainer.fit(model=model, datamodule=datamodule)
    
        test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc"]
        z, y = trainer.predict(model=model, datamodule=datamodule)[0]
        z, y = z.cpu(), y.cpu()
    
        fig = visualize_embeddings(z=z, y=y)
        fig.suptitle(f"GBT base {model_name} - test AUC: {test_auc * 100.:.2f} [%]")
    
        plt.show()
    

evaluate_gbt_models()

## Zadanie 3.3 (2 pkt)
Zbadaj wpływ hiperparametrów maskowania atrybutów wierzchołków `p_x` oraz usuwania krawędzi `p_e` na jakość reprezentacji:
- wybierz kilka (min. 4) wartości dla każdego hiperparametru
- zewaluuj model dla iloczynu kartezjańskiego tych hiperparametrów
- każdy eksperyment powtórz min. 2 razy
- w tabelce lub na wykresie przedstaw jak dane hiperparametry wpływały na jakość reprezentacji w zadaniu docelowym

In [ ]:
# TU WPISZ KOD
raise NotImplementedError()

## Zadanie 3.4 (1 pkt)
Zbadaj wpływ normalizacji `BatchNorm` na jakość reprezentacji:
- korzystając z wyników z poprzedniego zadania, wybierz najlepszy zestaw hiperparametrów `p_x` i `p_e`
- zewaluuj model z użyciem kodera GNNowego, który nie używa batch normalizacji 
- każdy eksperyment powtórz min. 2 razy
- w tabelce lub na wykresie przedstaw jak użyte kodery wpływały na jakość reprezentacji w zadaniu docelowym

In [ ]:
# TU WPISZ KOD
raise NotImplementedError()